In [54]:
import Pkg; 
Pkg.add("FFTW");
Pkg.add("PaddedViews");
Pkg.add("BenchmarkTools");
Pkg.add("ChainRules");
Pkg.add("ChainRulesCore");

   Resolving package versions...
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
    Updating `C:\Users\zinli\.julia\environments\v1.8\Project.toml`
  [d360d2e6] + ChainRulesCore v1.15.7
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Manifest.toml`


In [57]:
using FFTW
using Base.Threads
using PaddedViews
using BenchmarkTools
using ChainRulesCore

#try out different fft planners https://docs.juliahub.com/FourierAnalysis/1aWDG/1.0.1/fftw/
#also switch to in-place operations at some point
function create_2Dffp_plans(n::Int64,nthr::Int64)
    FFTW.set_num_threads(nthr)
    plan_fwd = plan_fft(rand(ComplexF64, n,n),flags=FFTW.MEASURE)
    plan_bwd = plan_ifft(rand(ComplexF64,n,n),flags=FFTW.MEASURE)
    (plan_fwd,plan_bwd)
end

function fftconv2d(arr::Array{ComplexF64,2},fftker::Array{ComplexF64,2},plan_fwd,plan_bwd,fwd::Bool=true)::Array{ComplexF64,2}
    narr = size(arr)[1]
    nker = size(fftker)[1]
    nout = nker - narr
    if fwd==true
        i1,i2 = 1+narr÷2, narr÷2+nout
    else
        i1,i2 =   narr÷2, narr÷2+nout-1
    end
    ifftshift(plan_bwd * (fftker .* (plan_fwd * sym_paddedviews(0.0+im*0.0,arr,fftker)[1])))[i1:i2,i1:i2]
end
    
function green2d(nx::Int64,ny::Int64, δx::Float64, δy::Float64, freq::Float64, ϵ::Float64,μ::Float64, Dz::Float64, plan)::Tuple{Array{ComplexF64,2},Array{ComplexF64,2}}

    ω = 2*π*freq
    n = sqrt(ϵ*μ)
    k = n*ω
    ik = im * k

    Lx,Ly = nx*δx, ny*δy
    δxy = δx*δy

    x = range(-Lx/2,Lx/2-δx, nx)'
    y = range(-Ly/2,Ly/2-δy, ny)

    gz = @. Dz * (-1 + ik * sqrt(x^2 + y^2 + Dz^2)) * cis(k*sqrt(x^2 + y^2 + Dz^2))/(4*π*sqrt(x^2 + y^2 + Dz^2)^3) * δxy * (-μ/ϵ)

    fgz = plan * gz
    fgzT = plan * reverse(gz)

    (fgz,fgzT)

end

# Field matrix F has the format (unit cells, frequencies)
function ffp(F::Array{ComplexF64,2}, fgs::Vector{Tuple{Array{ComplexF64,2},Array{ComplexF64,2}}}, plan_fwd, plan_bwd, fwd::Bool)::Array{ComplexF64,2}
    
    narr = Int64(sqrt(size(F)[1]))
    nker = size(fgs)[1]
    nout = nker - narr
    nfreqs = size(F)[2]

    out = zeros(ComplexF64, nout*nout, nfreqs)
    Threads.@threads for i in 1:nfreqs
        arr = reshape(F[:,i],narr,narr)
        if fwd==true
            fftker = fgs[i][1]
        else
            fftker = fgs[i][2]
        out[:,i] = vec(fftconv2d(arr,fftker, plan_fwd,plan_bwd, fwd))
        end
    end
    out
end

function near2far(F::Array{ComplexF64,2}, fgs::Vector{Tuple{Array{ComplexF64,2},Array{ComplexF64,2}}}, plan_fwd, plan_bwd)::Array{ComplexF64,2}
    ffp(F,fgs,plan_fwd,plan_bwd,true)
end
    

function ChainRulesCore.rrule(::typeof(near2far), F::Array{ComplexF64,2}, fgs::Vector{Tuple{Array{ComplexF64,2},Array{ComplexF64,2}}}, plan_fwd, plan_bwd)
    efar = near2far(F,fgs,plan_fwd,plan_bwd)
    function near2far_pullback(vec::Array{ComplexF64,2})
        dF = @thunk(ffp(vec, fgs, plan_fwd, plan_bwd, false))
        NoTangent(), dF, ZeroTangent(), ZeroTangent(), ZeroTangent()
    end
    efar, near2far_pullback
end

In [42]:
(p1,p2) = create_2Dffp_plans(500,1);
@btime (fgz,fgzT) = green2d(500,500,0.7,0.7, 1.0,1.0,1.0, 1000., p1)
display(fgz)
typeof(fgz)

  11.210 ms (9 allocations: 15.26 MiB)


500×500 Matrix{ComplexF64}:
 -0.522172+0.0482458im     0.500499-0.0138513im   …     0.500499-0.0138513im
  0.500499-0.0138513im    -0.475831-0.0246998im        -0.475831-0.0246998im
 -0.516946+0.100563im      0.499182-0.0639106im         0.499182-0.0639106im
  0.482015-0.11712im      -0.465475+0.0746756im        -0.465475+0.0746756im
 -0.469795+0.250476im      0.464862-0.209647im          0.464862-0.209647im
  0.384693-0.301741im     -0.385906+0.256922im    …    -0.385906+0.256922im
 -0.299333+0.446769im       0.31659-0.408237im           0.31659-0.408237im
  0.123852-0.469806im     -0.149657+0.435093im         -0.149657+0.435093im
 0.0502312+0.532301im   -0.00998056-0.514344im       -0.00998056-0.514344im
 -0.280429-0.410333im      0.239471+0.407105im          0.239471+0.407105im
  0.431096+0.281981im     -0.390649-0.303037im    …    -0.390649-0.303037im
 -0.521939+0.0303823im     0.501279+0.00417598im        0.501279+0.00417598im
  0.410391-0.264421im     -0.410204+0.219757im        

Matrix{ComplexF64} (alias for Array{Complex{Float64}, 2})

In [2]:
arr = rand(ComplexF64,5000,5000)
fftker = rand(ComplexF64,8000,8000)

(p1,p2) = create_2Dffp_plans(8000,2); @btime outarr = fftconv2d(arr,fftker,p1,p2,true);
(p1,p2) = create_2Dffp_plans(8000,4); @btime outarr = fftconv2d(arr,fftker,p1,p2,true);

  2.788 s (139 allocations: 4.90 GiB)
  2.426 s (194 allocations: 4.90 GiB)


In [3]:
typeof(p1)

FFTW.cFFTWPlan{ComplexF64, -1, false, 2, UnitRange{Int64}}

In [37]:
x = range(1,3,3)'
y = range(1,3,3)
display(x)
display(y)
@. sqrt(x^2 + y^2 + 1)

1×3 adjoint(::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}) with eltype Float64:
 1.0  2.0  3.0

1.0:1.0:3.0

3×3 Matrix{Float64}:
 1.73205  2.44949  3.31662
 2.44949  3.0      3.74166
 3.31662  3.74166  4.3589

In [48]:
x=9
Int64(sqrt(x))

3